In [1]:
import pandas as pd
from typing import Optional
import numpy as np

In [2]:
caminho_arquivo = "data/CPU_anon.csv"

In [3]:
df1 = pd.read_csv(caminho_arquivo, sep=",", header=0)

In [4]:
df1.head()

,Time,pod_1,pod_2,pod_3,pod_4,pod_5,pod_6,pod_7,pod_8,pod_9,...,pod_45,pod_46,pod_47,pod_48,pod_49,pod_50,pod_51,pod_52,pod_53,pod_54
0,2025-10-09 23:15:00,0.0470,NaN,3.02,0.01580,0.893,0.452,0.00102,0.147,0.153,...,0.0128,0.0137,0.988,0.00101,2.06,2.43,0.00245,NaN,NaN,NaN
1,2025-10-09 23:20:00,0.0528,NaN,2.77,0.01470,1.040,0.456,0.00103,0.130,0.143,...,0.0125,0.0143,0.990,0.00100,1.55,2.46,0.00246,NaN,NaN,NaN
2,2025-10-09 23:25:00,0.0460,NaN,3.27,0.00307,1.030,0.461,0.00102,0.129,0.143,...,0.0126,0.0112,0.988,0.00100,1.33,2.64,0.00251,NaN,NaN,NaN
3,2025-10-09 23:30:00,0.0455,NaN,3.21,0.00263,0.906,0.458,0.00104,0.127,0.139,...,0.0127,0.0118,0.993,0.00101,1.12,2.62,0.00253,NaN,NaN,NaN
4,2025-10-09 23:35:00,0.0568,NaN,3.03,0.00873,1.020,0.471,0.00104,0.138,0.149,...,0.0125,0.0215,0.985,0.00102,2.48,1.99,0.00252,NaN,NaN,NaN


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2017 entries, 0 to 2016
Data columns (total 55 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    2017 non-null   object 
 1   pod_1   2017 non-null   float64
 2   pod_2   388 non-null    float64
 3   pod_3   2017 non-null   float64
 4   pod_4   2017 non-null   float64
 5   pod_5   2017 non-null   float64
 6   pod_6   2015 non-null   float64
 7   pod_7   2017 non-null   float64
 8   pod_8   2017 non-null   float64
 9   pod_9   2017 non-null   float64
 10  pod_10  2017 non-null   float64
 11  pod_11  2017 non-null   float64
 12  pod_12  2017 non-null   float64
 13  pod_13  2013 non-null   float64
 14  pod_14  512 non-null    float64
 15  pod_15  2016 non-null   float64
 16  pod_16  2017 non-null   float64
 17  pod_17  388 non-null    float64
 18  pod_18  2017 non-null   float64
 19  pod_19  2017 non-null   float64
 20  pod_20  2017 non-null   float64
 21  pod_21  2017 non-null   float64
 22  

In [6]:
def consolidar_memoria_por_workload(
    df: pd.DataFrame,
    time_col: str = "Time",
    dayfirst: bool = True,
    ultimos_dias: int = 7,
    valor_colname: str = "memory_value",
    workload_colname: str = "workload",
) -> pd.DataFrame:
    """
    Consolida estatísticas de uso de memória por workload para os últimos `ultimos_dias`.

    Passos:
      1) Converte a coluna de tempo e filtra últimos N dias (com base no max(Time)).
      2) Converte o DataFrame de wide -> long (melt), criando colunas [Time, workload, memory_value].
      3) Calcula, por workload: min, max, mean, mode e p95.
      4) Retorna uma tabela com uma linha por workload (pivot consolidado).

    Parâmetros
    ----------
    df : pd.DataFrame
        DataFrame no formato wide (coluna 'Time' + colunas de workloads).
    time_col : str
        Nome da coluna de tempo.
    dayfirst : bool
        Se True, interpreta datas no formato dia/mês/ano.
    ultimos_dias : int
        Janela temporal para consolidação (dias a partir do max(Time)).
    valor_colname : str
        Nome da coluna de valores na versão long.
    workload_colname : str
        Nome da coluna que conterá o nome do workload na versão long.

    Retorna
    -------
    pd.DataFrame
        Tabela consolidada com colunas:
        [workload, count, min, max, mean, mode, p95]
    """

    # 1) Tempo: parse e filtro últimos N dias
    if df[time_col].dtype != "datetime64[ns]":
        df = df.copy()
        df[time_col] = pd.to_datetime(df[time_col], dayfirst=dayfirst, errors="coerce")

    # Remove registros sem tempo interpretável
    df = df.dropna(subset=[time_col])

    # Determina janela dos últimos N dias em relação ao max(Time)
    fim = df[time_col].max()
    ini = fim - pd.Timedelta(days=ultimos_dias)
    df7 = df[(df[time_col] >= ini) & (df[time_col] <= fim)].copy()

    # 2) Wide -> Long (melt): todas as colunas menos 'Time' viram uma coluna 'workload'
    valor_cols = [c for c in df7.columns if c != time_col]
    long_df = df7.melt(
        id_vars=[time_col],
        value_vars=valor_cols,
        var_name=workload_colname,
        value_name=valor_colname,
    )

    # Mantém apenas valores numéricos
    long_df[valor_colname] = pd.to_numeric(long_df[valor_colname], errors="coerce")
    long_df = long_df.dropna(subset=[valor_colname])

    # 3) Agregações
    def _mode_first(s: pd.Series) -> Optional[float]:
        m = s.mode(dropna=True)
        return m.iloc[0] if not m.empty else pd.NA

    agg_df = long_df.groupby(workload_colname)[valor_colname].agg(
        count="count",
        min="min",
        max="max",
        mean="mean",
        mode=_mode_first,
        p95=lambda x: x.quantile(0.95, interpolation="linear"),
    )

    # 4) Organização final
    agg_df = agg_df.reset_index().sort_values("mean", ascending=False)

    return agg_df



In [7]:
tabela_consolidada = consolidar_memoria_por_workload(df1)

In [8]:
tabela_consolidada = tabela_consolidada.rename(columns={
    "count": "Registros",
    "min": "CPUMin",
    "max": "CPUMax",
    "mean": "CPUMédia",
    "mode": "CPUModa",
    "p95": "CPUP95"
})

In [9]:
tabela_consolidada.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, 42 to 9
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   workload   48 non-null     object 
 1   Registros  48 non-null     int64  
 2   CPUMin     48 non-null     float64
 3   CPUMax     48 non-null     float64
 4   CPUMédia   48 non-null     float64
 5   CPUModa    48 non-null     float64
 6   CPUP95     48 non-null     float64
dtypes: float64(5), int64(1), object(1)
memory usage: 3.0+ KB


In [10]:
tabela_consolidada.head(100)

,workload,Registros,CPUMin,CPUMax,CPUMédia,CPUModa,CPUP95
42,pod_50,288,3.750000,7.460000,5.929097,5.740000,6.922000
19,pod_3,288,1.480000,4.870000,3.374931,3.900000,4.716500
2,pod_11,288,2.070000,3.940000,2.474549,2.300000,3.423000
38,pod_47,288,0.895000,0.985000,0.958639,0.963000,0.977000
41,pod_5,288,0.320000,1.340000,0.918028,1.190000,1.290000
44,pod_6,288,0.506000,1.180000,0.815837,0.897000,1.100000
40,pod_49,288,0.135000,5.820000,0.593326,0.254000,1.336000
47,pod_9,288,0.294000,0.435000,0.380306,0.391000,0.417000
34,pod_43,288,0.157000,0.498000,0.284670,0.168000,0.464000
46,pod_8,288,0.157000,0.312000,0.193823,0.172000,0.296000


In [11]:
caminho_arquivo = "data/MEM_anon.csv"

In [12]:
df2 = pd.read_csv(caminho_arquivo, sep=",", header=0)

In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2017 entries, 0 to 2016
Data columns (total 55 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    2017 non-null   object 
 1   pod_1   2017 non-null   object 
 2   pod_2   389 non-null    object 
 3   pod_3   2017 non-null   object 
 4   pod_4   2017 non-null   object 
 5   pod_5   2017 non-null   object 
 6   pod_6   2017 non-null   object 
 7   pod_7   2017 non-null   object 
 8   pod_8   2017 non-null   object 
 9   pod_9   2017 non-null   object 
 10  pod_10  2017 non-null   object 
 11  pod_11  2017 non-null   object 
 12  pod_12  2017 non-null   object 
 13  pod_13  2017 non-null   object 
 14  pod_14  514 non-null    object 
 15  pod_15  2017 non-null   object 
 16  pod_16  2017 non-null   object 
 17  pod_17  389 non-null    object 
 18  pod_18  2017 non-null   object 
 19  pod_19  2017 non-null   object 
 20  pod_20  47 non-null     object 
 21  pod_21  2017 non-null   object 
 22  

In [14]:


def consolidar_df2_com_objetos(
    df2: pd.DataFrame,
    time_col: str = "Time",
    ultimos_dias: int = 7,
    dayfirst: bool = True,
) -> pd.DataFrame:
    """
    Consolida estatísticas (min, max, média, moda, p95) de uso de memória por workload.
    Adaptada para DataFrames em que as colunas vêm como 'object' (strings).

    Parâmetros:
    -----------
    df2 : pd.DataFrame
        DataFrame com colunas de workloads e uma coluna 'Time'.
    time_col : str
        Nome da coluna com os registros de tempo.
    ultimos_dias : int
        Quantos dias anteriores considerar no cálculo.
    dayfirst : bool
        Define se as datas estão no formato dia/mês/ano (True por padrão).

    Retorna:
    --------
    pd.DataFrame
        Tabela consolidada com colunas:
        ['workload', 'count', 'min', 'max', 'mean', 'mode', 'p95']
    """

    df = df2.copy()

    # 1️) Converter a coluna Time para datetime
    df[time_col] = pd.to_datetime(df[time_col], dayfirst=dayfirst, errors="coerce")
    df = df.dropna(subset=[time_col])

    # 2️) Filtrar os últimos N dias
    fim = df[time_col].max()
    ini = fim - pd.Timedelta(days=ultimos_dias)
    df = df[(df[time_col] >= ini) & (df[time_col] <= fim)]

    # 3️) Selecionar colunas de workloads (descartando quotas e colunas vazias)
    valor_cols = [
        c for c in df.columns
        if c != time_col and not c.lower().startswith("quota")
    ]

    # 4️) Converter todos os valores para float
    def limpar_e_converter(col):
        # converte tudo para string
        s = col.astype(str).str.strip()
        # troca vírgula decimal por ponto
        s = s.str.replace(",", ".", regex=False)
        # extrai o primeiro número da string (descarta texto tipo 'MiB')
        s = s.str.extract(r"([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?)", expand=False)
        # converte para float
        return pd.to_numeric(s, errors="coerce")

    df[valor_cols] = df[valor_cols].apply(limpar_e_converter)

    # 5️) Transformar de wide → long
    long_df = df.melt(
        id_vars=[time_col],
        value_vars=valor_cols,
        var_name="workload",
        value_name="memory_value"
    ).dropna(subset=["memory_value"])

    # 6️) Calcular agregados
    def _mode_first(s: pd.Series) -> Optional[float]:
        m = s.mode(dropna=True)
        return m.iloc[0] if not m.empty else np.nan

    agg = (
        long_df.groupby("workload")["memory_value"]
        .agg(
            count="count",
            min="min",
            max="max",
            mean="mean",
            mode=_mode_first,
            p95=lambda x: x.quantile(0.95, interpolation="linear")
        )
        .reset_index()
        .sort_values("mean", ascending=False)
    )

    return agg

In [15]:
tabela = consolidar_df2_com_objetos(df2)

In [16]:
tabela = tabela.rename(columns={
    "count": "Registros",
    "min": "MEMMin",
    "max": "MEMMax",
    "mean": "MEMMédia",
    "mode": "MEMModa",
    "p95": "MEMP95"
})

In [17]:
tabela.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, 30 to 47
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   workload   48 non-null     object 
 1   Registros  48 non-null     int64  
 2   MEMMin     48 non-null     float64
 3   MEMMax     48 non-null     float64
 4   MEMMédia   48 non-null     float64
 5   MEMModa    48 non-null     float64
 6   MEMP95     48 non-null     float64
dtypes: float64(5), int64(1), object(1)
memory usage: 3.0+ KB


In [18]:
tabela.head()

,workload,Registros,MEMMin,MEMMax,MEMMédia,MEMModa,MEMP95
30,pod_4,288,1.01,1021.0,711.921076,1.08,1012.00
35,pod_44,288,1.11,859.0,696.034444,700.00,752.65
1,pod_10,288,638.00,748.0,692.979167,638.00,742.65
39,pod_48,288,415.00,449.0,448.409722,449.00,449.00
34,pod_43,288,409.00,417.0,413.000000,412.00,416.00


In [19]:
tabela_final = pd.merge(
    tabela,
    tabela_consolidada,
    on="workload", 
    how="inner" 
)


In [20]:
tabela_final = pd.merge(
    tabela,
    tabela_consolidada,
    on="workload",
    how="inner",
    suffixes=("", "_drop")   # evita duplicação visual
)

# Remove a coluna duplicada, se existir
if "Registros_drop" in tabela_final.columns:
    tabela_final = tabela_final.drop(columns=["Registros_drop"])

In [21]:
tabela_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   workload   48 non-null     object 
 1   Registros  48 non-null     int64  
 2   MEMMin     48 non-null     float64
 3   MEMMax     48 non-null     float64
 4   MEMMédia   48 non-null     float64
 5   MEMModa    48 non-null     float64
 6   MEMP95     48 non-null     float64
 7   CPUMin     48 non-null     float64
 8   CPUMax     48 non-null     float64
 9   CPUMédia   48 non-null     float64
 10  CPUModa    48 non-null     float64
 11  CPUP95     48 non-null     float64
dtypes: float64(10), int64(1), object(1)
memory usage: 4.6+ KB


In [22]:
tabela_final.to_csv("tabela_final.csv", index=False)

In [23]:
tabela_final.head(78)

,workload,Registros,MEMMin,MEMMax,MEMMédia,MEMModa,MEMP95,CPUMin,CPUMax,CPUMédia,CPUModa,CPUP95
0,pod_4,288,1.01,1021.00,711.921076,1.08,1012.000,0.001870,0.027900,0.019203,0.021300,0.026200
1,pod_44,288,1.11,859.00,696.034444,700.00,752.650,0.000195,0.998000,0.026034,0.000221,0.000229
2,pod_10,288,638.00,748.00,692.979167,638.00,742.650,0.004300,0.004970,0.004681,0.004640,0.004890
3,pod_48,288,415.00,449.00,448.409722,449.00,449.000,0.001050,0.002540,0.001205,0.001090,0.002283
4,pod_43,288,409.00,417.00,413.000000,412.00,416.000,0.157000,0.498000,0.284670,0.168000,0.464000
5,pod_42,288,1.02,439.00,283.411701,438.00,439.000,0.000768,0.474000,0.052613,0.000802,0.262000
6,pod_51,288,246.00,246.00,246.000000,246.00,246.000,0.002420,0.002690,0.002595,0.002600,0.002660
7,pod_30,288,1.07,878.00,232.473854,219.00,220.000,0.000715,0.481000,0.040421,0.000816,0.312550
8,pod_18,288,1.01,1020.00,228.758229,214.00,334.850,0.000762,0.527000,0.048726,0.000816,0.365300
9,pod_41,288,1.04,1012.00,202.649444,213.00,583.300,0.000778,0.437000,0.050788,0.000863,0.254250


In [24]:
# 1️) Extrai a parte depois do " - "   e guarda na nova coluna
tabela_final["tipo"] = tabela_final["workload"].str.extract(r" -\s*(.*)$")

# 2️) Remove o sufixo ' - ...' da coluna original
tabela_final["workload"] = tabela_final["workload"].str.replace(r" -\s*.*$", "", regex=True)

# 3️) Limpa espaços extras
tabela_final["workload"] = tabela_final["workload"].str.strip()
tabela_final["tipo"] = tabela_final["tipo"].str.strip()

print(tabela_final[["workload", "tipo"]].head())

  workload tipo
0    pod_4  NaN
1   pod_44  NaN
2   pod_10  NaN
3   pod_48  NaN
4   pod_43  NaN


In [25]:
cols = ["workload", "tipo"] + [c for c in tabela_final.columns if c not in ["workload", "tipo"]]
tabela_final = tabela_final[cols]

In [26]:
nova_ordem = [
    "workload",
    "Registros",
    "tipo",
    "CPUMin", "CPUMax", "CPUMédia", "CPUModa", "CPUP95",
    "MEMMin", "MEMMax", "MEMMédia", "MEMModa", "MEMP95"
]


tabela_final = tabela_final[nova_ordem]

In [27]:
tabela_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   workload   48 non-null     object 
 1   Registros  48 non-null     int64  
 2   tipo       0 non-null      object 
 3   CPUMin     48 non-null     float64
 4   CPUMax     48 non-null     float64
 5   CPUMédia   48 non-null     float64
 6   CPUModa    48 non-null     float64
 7   CPUP95     48 non-null     float64
 8   MEMMin     48 non-null     float64
 9   MEMMax     48 non-null     float64
 10  MEMMédia   48 non-null     float64
 11  MEMModa    48 non-null     float64
 12  MEMP95     48 non-null     float64
dtypes: float64(10), int64(1), object(2)
memory usage: 5.0+ KB


In [28]:
tabela_final.to_excel("calculoCPUMemoriaWorkloads.xlsx", index=False, sheet_name="Consolidado")